In [7]:
import os 
import sys

In [8]:
# Set path to parent dir to import personal imports
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from data.TrajectorySet import TrajectorySet
from data.Sampler import Sampler 
from utils.tensor_utils import convert_batch_to_tensor

import torch 
import torch.utils.data as data 
from torch.utils.data import Dataset

import minari 

In [9]:
minari_dataset = minari.load_dataset("D4RL/pointmaze/large-v2")

In [10]:
class DatasetCL(Dataset): 
    def __init__(self, sampler, batch_size: int, k: int = 2): 
        """
        sampler: The Sampler class to sample batches. 
        batch_size: The size of the batch (ie. the number of state pairs)
        k: A hyperparameter that dictates the average number of 
                positive pairs sampled from the same trajectory. The 
                lower the number, the lesser the chance of false negatives. 
        """
        self.sampler = sampler 
        self.batch_size = batch_size 
        self.k = k 

        self.batch = convert_batch_to_tensor(self.sampler.sample_batch(batch_size=self.batch_size, k=self.k))
    
    def __len__(self): 
        return len(self.batch[0])
    
    def __getitem__(self, index):
        s_i = self.batch[0][index] 
        s_j = self.batch[1][index] 
        return torch.tensor(s_i, dtype=torch.float32), torch.tensor(s_j, dtype=torch.float32)
    
    def get_batch(self):
        return self.batch

In [12]:
"""
TESTING CELL! 
"""

T = TrajectorySet(dataset=minari_dataset) 

S = Sampler(T, dist="u") 
ds = DatasetCL(sampler=S, batch_size=4, k=4)


batch = ds.get_batch()
print(batch)
#print(len(ds))
#print(ds[0])


DS = data.DataLoader(dataset=ds, batch_size=4, shuffle=True)
print("HELLOO")
print(len(DS))
DS_ = iter(DS)
for i in range(len(DS)): 
    print(next(DS_))



(tensor([[ 3.3062, -1.1341, -4.0309,  0.3605],
        [ 4.5005, -0.6255,  0.2590, -4.7499],
        [ 3.3465, -1.1377, -3.8018,  0.1993],
        [ 2.5758, -1.0720, -3.5289, -0.0272]]), tensor([[ 4.5007, -0.6721,  0.0202, -4.6657],
        [ 2.4449, -1.0784, -3.4977, -0.4543],
        [ 3.8298, -1.1400, -5.0101,  0.1559],
        [ 3.3846, -1.1397, -3.5721,  0.0892]]))
HELLOO
1
[tensor([[ 2.5758, -1.0720, -3.5289, -0.0272],
        [ 3.3465, -1.1377, -3.8018,  0.1993],
        [ 3.3062, -1.1341, -4.0309,  0.3605],
        [ 4.5005, -0.6255,  0.2590, -4.7499]]), tensor([[ 3.3846, -1.1397, -3.5721,  0.0892],
        [ 3.8298, -1.1400, -5.0101,  0.1559],
        [ 4.5007, -0.6721,  0.0202, -4.6657],
        [ 2.4449, -1.0784, -3.4977, -0.4543]])]


/var/folders/89/s23t762151989wcy_1bx9vqc0000gn/T/ipykernel_59948/2349861161.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(s_i, dtype=torch.float32), torch.tensor(s_j, dtype=torch.float32)
